In [3]:
!pip install -qq gradio openai langchain pydub

In [4]:
import gradio as gr
from openai import OpenAI
import openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from google.colab import userdata

open_api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = open_api_key

음성인식 봇 코드

In [5]:
system_ai_ko= "당신은 인공지능 비서야, pdf 읽기, 문서요약하기, 일정 관리, 날씨, 최단경로 검색, 웹 검색 등 다양한 내용에 답변할 수 있어야해"
system_setting = SystemMessagePromptTemplate.from_template(system_ai_ko)
# 프롬프트 설정
voice_bot_prompt = ChatPromptTemplate.from_messages([
    system_setting,
    MessagesPlaceholder(variable_name="HeyMate"),
    HumanMessagePromptTemplate.from_template("{master_user}"),])

In [6]:
# 메모리 설정
voice_bot_memory = ConversationBufferWindowMemory(
    memory_key="HeyMate",
    ai_prefix="AI 비서 HeyMate",
    human_prefix="사용자",
    return_messages=True,
    k=10)

In [7]:
# llm 모델 정의
chatgpt = ChatOpenAI(
    temperature=0.3,
    max_tokens=2048,
    model_name="gpt-3.5-turbo"
    )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
# llmchain 정의
conversation = LLMChain(
prompt=voice_bot_prompt,
        memory=voice_bot_memory,
llm=chatgpt,
verbose=True,  # True 로 설정시 로그 출력
    )

In [9]:
from pydub import AudioSegment
def voice_bot_handle_audio(audio_record):
    save_file_path = "voice.mp3"
    frame_rate = audio_record[0]
    audio_data = audio_record[1].tobytes()
    sample_width =audio_record[1].dtype.itemsize
    audio = AudioSegment(
        audio_data,
        frame_rate=frame_rate,
        sample_width=sample_width,
        channels=1,
    )
    audio.export(save_file_path, format="mp3")

In [10]:
#음성 입력 텍스트 변환하기
def voice_bot_create_stt():
    client = OpenAI()
    file_path = "voice.mp3"
    audio_file = open(file_path, "rb")
    transcript = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file)
    return transcript.text

In [11]:
#인공지능 답변 음성 변환하기
def voice_bot_create_audio(text):
    client = OpenAI()
    response = client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=text)
    speech_file_path = "output.mp3"  # 오디오 파일 저장 위치
    response.stream_to_file(speech_file_path)
    return speech_file_path  # 오디오 파일의 경로를 반환

In [12]:
#음성 인식봇 음성 답변 구현하기
def voice_bot_chat(message, cb_user_input_audio, chat_history):
    if cb_user_input_audio:
        message = voice_bot_create_stt()
    ai_answer = conversation({"master_user": message})['text']
    chat_history.append((message, ai_answer))
    audio_file = voice_bot_create_audio(ai_answer)  # 오디오 파일 생성
    return "", audio_file, chat_history  # 오디오 파일 경로와 채팅 기록 반환

In [13]:
def voice_bot_undo(chat_history):
    if len(chat_history) > 1:
        chat_history.pop()
    return chat_history

In [14]:
# 챗봇 초기화
def voice_bot_reset(chat_history):
    global voice_bot_memory
    chat_init = [[None, '안녕하세요 AI 인공지능 비서 HeyMate 입니다. 무엇이든 시켜만 주세요']]
    voice_bot_memory.clear() # 메모리 초기화
    return "", chat_init

문서요약 봇 코드

In [15]:
!pip install -qq pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00


In [16]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import MapReduceDocumentsChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [17]:
def pdf_loader(pdf_path):
    loader = PyPDFLoader(pdf_path)
    pdf_doc = loader.load()
    return pdf_doc

In [18]:
def pdf_bot_chatbot(pdf_path):
    pages_content = pdf_loader(pdf_path)
    # 스플리터는 설정
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=3000,
        chunk_overlap=0,
    )
    # pages_content 내용 분할
    split_docs = text_splitter.split_documents(pages_content)
    # 분할된 문서의 수
    # Map template 설정, {pages_content} 분할된 내용이 입력
    map_template = """다음은 문서 중 일부 내용입니다
    {pages_content}
    이 문서의 주요 내용을 요약해 주세요.
    """
    # Map 기본 프롬프트
    map_prompt = PromptTemplate.from_template(map_template)
    # 문서내용이 길수 있기 때문에 model을 gpt-3.5-turbo-16k설정
    llm = ChatOpenAI(temperature=0,
                    model_name='gpt-3.5-turbo-16k')
    map_chain = LLMChain(llm=llm, prompt=map_prompt)
    # Reduce 단계에서 처리할 프롬프트 정의
    reduce_template = """다음은 문서 요약의 집합입니다:
    {summaries}
    이 내용을 바탕으로 통합된 문서 요약을 작성해 주세요.
    """
    # Reduce 프롬프트
    reduce_prompt = PromptTemplate.from_template(reduce_template)

    # Reduce에서 수행할 LLMChain 정의
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)
    from langchain.chains.combine_documents.stuff import StuffDocumentsChain
    from langchain.chains import ReduceDocumentsChain

    # 문서의 목록을 통합하는 체인
    combine_doc_chain = StuffDocumentsChain(
        llm_chain=reduce_chain,
        document_variable_name="summaries" # Reduce 프롬프트에 대입되는 변수
    )

    # 분할된 문서 순차적으로 Reduce 처리
    reduce_doc_chain = ReduceDocumentsChain(
        combine_documents_chain=combine_doc_chain, # 최종 체인 - 문서 결합체인
        collapse_documents_chain=combine_doc_chain, # 필요한 경우 문서 축소 체인
        token_max=4000, # 토큰 최대 갯수 설정
    )

    # 최종 체인 연결
    final_chain = MapReduceDocumentsChain(
        llm_chain=map_chain, # 각 문서 매핑
        reduce_documents_chain=reduce_doc_chain, # 문서 통합
        document_variable_name="pages_content", # llm_cahin에 입력받을 문서명
        return_intermediate_steps=False, # 최종결과만
    )
    # 최종 결과 실행
    result_summary = final_chain.run(split_docs)
    # 요약결과 출력
    return result_summary

문서요약 봇 코드

In [19]:
# 프롬프트 정의
import json
system_schedule_ko = """일정 관리에 대한 사용자 입력이 주어집니다.
'schedule_type,' 'schedule_content,' 'schedule_content_detail,' 'year,' 'month,' 'day,' 'hour,' 및 'min'과 같은 구성 요소가 있습니다.
'schedule_type'은 조회, 삭제, 추가, 업데이트 중 하나일 수 있습니다. 'year,' 'month,' 'day,' 'hour,' 및 'min'에 대한 값은 숫자여야 합니다.
'schedule_content,' 'schedule_content_detail,' 'year,' 'month,' 'day,' 'hour,' 및 'min'에 대한 입력은 JSON 문자열 형식으로 이루어져야 합니다."""
system_setting_schedule = SystemMessagePromptTemplate.from_template(system_schedule_ko)
# 프롬프트 설정
schedule_prompt = ChatPromptTemplate.from_messages([
    system_setting_schedule,                                           # 역할부여
    MessagesPlaceholder(variable_name="HeyMate_schedule"),              # 메모리 저장소 설정. ConversationBufferMemory의 memory_key 와 동일하게 설정
    HumanMessagePromptTemplate.from_template("{master_user}"), # 사용자 메시지 injection
])

In [20]:
# 메모리 설정
schedule_memory = ConversationBufferWindowMemory(memory_key="HeyMate_schedule",
                                  ai_prefix="AI 비서 HeyMate schedule",
                                  human_prefix="사용자",
                                  return_messages=True,
                                  k=10)
# llm 모델 정의
chatgpt = ChatOpenAI(
    temperature=0.3,
    max_tokens=2048,
    model_name="gpt-3.5-turbo"
    )

# llmchain 정의
conversation_schedule = LLMChain(
        llm=chatgpt,   # LLM
        prompt=schedule_prompt, # Prompt
        verbose=True,  # True 로 설정시 로그 출력
        memory=schedule_memory  # 메모리
    )

In [21]:
# 일정 관리 excel 파일 관리
import pandas as pd
# 처음 파일 생성
initial_df = pd.DataFrame(columns=[
    "schedule_type",
    "schedule_content",
    "schedule_content_detail",
    "year",
    "month",
    "day",
    "hour",
    "min"
])
excel_file_path = "schedule.xlsx"
initial_df.to_excel(excel_file_path, index=False)
print(f"데이터를 저장할 {excel_file_path} 파일이 생성되었습니다.")

데이터를 저장할 schedule.xlsx 파일이 생성되었습니다.


In [22]:
# 엑셀파일 로드시 저장
def schedule_bot_save(submit_file):
    temp_excel_file = pd.read_excel(submit_file)
    temp_excel_file.to_excel(excel_file_path, index=False)

In [23]:
def schedule_bot_chat(message, chat_history):
    answer = conversation_schedule({"master_user": message})
    ai_answer = answer['text']
    print(ai_answer)

    try:
        schedule_dic = json.loads(ai_answer)
        if schedule_dic["schedule_type"] == "추가":
            schedule_df = pd.read_excel(excel_file_path)
            schedule_df = pd.concat([schedule_df, pd.DataFrame([schedule_dic])],
                                    ignore_index=True
                                    )
            schedule_df.to_excel(excel_file_path, index=False)
            chat_history.append([message,
                                 f"{schedule_dic['schedule_content']}_일정이 추가되었습니다."])

        elif schedule_dic["schedule_type"] == "조회":
            schedule_df = []
            if os.path.isfile(excel_file_path):
                schedule_df = pd.read_excel(excel_file_path)
            chat_history.append([message, "전체 일정을 말씀드리겠습니다."])
            for idx, event in schedule_df.iterrows():
                chat_history.append(
                    [None, f"{idx+1}. 일정: {event['schedule_content']}\n \
                    일정 설명: {event['schedule_content_detail']}\n \
                    일정 시간: {event['year']}년, {event['month']}월, {event['day']}일, {event['hour']}시, {event['min']}분"] )
    except:
        chat_history.append((message, ai_answer))
    return "", chat_history  # 채팅 기록 반환

gradio 코드

In [24]:
with gr.Blocks(theme=gr.themes.Default()) as app:
    with gr.Tab("음성 인식봇"):
        with gr.Row():
            gr.Markdown(
            value="""
            # <center>음성 인식 봇</center>
            <center>AI 인공지능 비서 HeyMate입니다. 음성으로 묻거나, 문서 요약, 일정 관리를 할 수 있습니다. </center>
            """)
        with gr.Row():
            cb_chatbot = gr.Chatbot(
                value = [
                    [None, "안녕하세요 AI 인공지능 비서 HeyMate 입니다. 무엇이든 시켜만 주세요"]
                ],
                show_label=False
            )
        with gr.Row():
            cb_user_input = gr.Textbox(
                lines=1,
                placeholder="입력 창",
                container=False,
                scale=7
            )
            cb_audio_record = gr.Audio(sources=["microphone"],
                                    format="mp3",
                                    scale=1,
                                    min_width=200,
                                    label="음성입력해주세요"
                                    )
            cb_audio_chatbot = gr.Audio(autoplay=True,
                                        visible=False)
            cb_submit_btn = gr.Button(
                value="보내기",
                scale=1,
                visible="primary",
                icon="https://cdn-icons-png.flaticon.com/128/12439/12439334.png"
            )
            # 음성생성으로
        cb_audio_record.stop_recording(voice_bot_handle_audio, cb_audio_record)
        cb_submit_btn.click(
            fn=voice_bot_chat,
            inputs=[cb_user_input, cb_audio_record, cb_chatbot],
            outputs=[cb_user_input, cb_audio_chatbot, cb_chatbot]
            )
        cb_user_input.submit(
            fn=voice_bot_chat,
            inputs=[cb_user_input, cb_audio_record, cb_chatbot],
            outputs=[cb_user_input, cb_audio_chatbot, cb_chatbot]
            )

        with gr.Row():
            gr.Button(value="되돌리기 ↩️").click(
                fn=voice_bot_undo,
                inputs=[cb_chatbot],
                outputs=[cb_chatbot]
                )
            gr.Button(value="초기화 🔄️").click(
                fn=voice_bot_reset,
                inputs=[cb_chatbot],
                outputs=[cb_chatbot]
                )
    with gr.Tab("PDF 문서 요약 봇"):
        with gr.Row():
            gr.Markdown(
            value="""
            # <center>PDF 요약 봇</center>
            <center>AI 인공지능 비서 HeyMate입니다. PDF 업로드하면 내용을 요약해 줄 수 있습니다.</center>
            """)
        with gr.Row():
            pdf_input = gr.File()
        with gr.Row():
            summary_btn = gr.Button(value="문서요약하기")
        with gr.Row():
            summary = gr.Textbox(
                label="PDF 요약",
                lines=8,
                placeholder="PDF 요약 내용입니다.",
                scale=8
                )
        summary_btn.click(pdf_bot_chatbot, inputs=pdf_input, outputs=summary)
    with gr.Tab("일정 관리봇"):
        with gr.Row():
            gr.Markdown(
                value="""
                # <center>일정관리 봇</center>
                <center>AI 인공지능 비서 HeyMate입니다. 일정 관리를 위한 챗봇입니다</center>
                """
                )
        with gr.Row():
            chatbot_schedule = gr.Chatbot(
                value=[
                    [None, "안녕하세요, 일정 이름, 시간, 일정 설명으로 일정을 추가할수 있습니다. \n\
                    예시: 크리스마스, 2023년 12월 25일 12시 00분, 올해의 크리스마스 일정 추가해 줘\n\
                    전체 일정이 보고싶다면 전체 일정 보여 줘라고 말해 주세요"],
                    ],
                show_label=False,
            )
        with gr.Row():
            msg_schedule = gr.Textbox(
                label="채팅",
                lines=1,
                placeholder="채팅 입력 창",
                scale=8,
                )
            cb_schedule_submit_btn = gr.Button(
                value="보내기",
                scale=2,
                visible="primary",
                )
        with gr.Row():
            schedule_file = gr.File(
                label="일정파일을 업로드해주세요",
                scale=8,
                visible="primary",
                height=100
                )
        cb_schedule_submit_btn.click(
            fn=schedule_bot_chat,
            inputs=[msg_schedule, chatbot_schedule],
            outputs=[msg_schedule, chatbot_schedule]
            )
        msg_schedule.submit(
            fn=schedule_bot_chat,
            inputs=[msg_schedule, chatbot_schedule],
            outputs=[msg_schedule, chatbot_schedule]
            )
        schedule_file.change(
            fn=schedule_bot_save,
            inputs=[schedule_file]
            )
app.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9ba0206267653d630d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9ba0206267653d630d.gradio.live
